In [6]:
import random
from DB import DB
import pandas as pd
from string import ascii_letters
from names import get_first_name

### Initialize a connection to database
In order to insert data to the database, a connection should be initialized between them.

In [7]:
database = DB().connection

### Load mock data
Mock data that is used here, could be found via [this link](https://www.kaggle.com/datasets/keshavramaiah/hotel-recommendation).
Data should be downloaded and stored in the "mock_data" folder.

In [8]:
hotels_df = pd.read_csv('mock_data/Hotel_details.csv')
rooms_df = pd.read_csv('mock_data/hotels_RoomPrice.csv')

### Prepare data which is related to the "cities" table

In [9]:
cities = hotels_df['city']
cities = cities.drop_duplicates()
cities = cities.reset_index()
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25693 entries, 0 to 25692
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   25693 non-null  int64 
 1   city    25693 non-null  object
dtypes: int64(1), object(1)
memory usage: 401.6+ KB


### Initialize a connection with the database

In [10]:
await database.connect()

### Insert Cities to the database
This block inserts the cities into a table that is called "cities" in the database

In [11]:
query = "INSERT INTO cities(id, name, image) VALUES"
image_link = 'https://unsplash.com/photos/Nyvq2juw4_o/download?ixid=MnwxMjA3fDB8MXxzZWFyY2h8Mnx8Y2l0eXxlbnwwfHx8fDE2NjUwMDAzNjM&force=true&w=640'
for index, city in cities.iterrows():
    query += "({}, '{}', '{}'),".format(str(index), city['city'].replace("'", "''"), image_link)
query = query[:-1]
await database.execute(query=query)

ConnectionDoesNotExistError: connection was closed in the middle of operation

### Prepare data which is relate to the "hotels" table

In [ ]:
hotels = hotels_df[['hotelid', 'hotelname', 'starrating', 'address', 'city']]
hotels = hotels.dropna()
hotels = hotels.drop_duplicates(subset=["hotelid"], keep='last')
hotels = hotels.reset_index()
hotels.info()

### Insert Hotels to the database
This block inserts the hotels into a table that is called "hotels" in the database

In [ ]:
load_in_each_query = 10000
for x in range(0, hotels['index'].count(), load_in_each_query):
    query = "INSERT INTO hotels(id, name, address, image, credit, star, city_id) VALUES "
    for index, hotel in hotels.iterrows():
        if x<=index<x+load_in_each_query:
            query += "({}, '{}', '{}', '{}', {}, {}, {}),".format(str(hotel['hotelid']), hotel['hotelname'].replace("'", "''"), hotel['address'].replace("'", "''"), 'image_link', 0, str(hotel["starrating"]), str(cities.loc[cities['city']==hotel['city']].index[0]))
    query = query[:-1]
    await database.execute(query=query)

### Prepare data which is relate to the "rooms" table

In [ ]:
rooms = rooms_df[['id', 'onsiterate', 'maxoccupancy', 'hotelcode']]
rooms = rooms.dropna()
rooms = rooms.reset_index()
rooms.info()

### Insert Rooms to the database
This block inserts the rooms into a table that is called "rooms" in the database

In [ ]:
load_in_each_query = 10000
for x in range(0, rooms['index'].count(), load_in_each_query):
    query = "INSERT INTO rooms(id, number, price, max_capacity, floor, hotel_id) VALUES "
    for index, room in rooms.iterrows():
        if x<=index<x+load_in_each_query:
            query += "({}, '{}', {}, {}, {}, {}),".format(str(index), str(int(room['id'])), str(int(room['onsiterate'])), str(room["maxoccupancy"]), 1, 8992131)
    query = query[:-1]
    await database.execute(query=query)

### Generate fake users

In [ ]:
query = "INSERT INTO users(id, name, email, email_verified_at, password, remember_token) VALUES "
for index in range(1, 300001):
    name=get_first_name()
    email=''
    for _ in range(10):
        email+=random.choice(ascii_letters)
    query += "({}, '{}', '{}', '{}', '{}', '{}'),".format(str(index), name, email+'@example.com', '2022-12-15 22:00:00', 'password', 'xzasfsfsfg')
    if index % 1000 == 0:
        query = query[:-1]
        await database.execute(query=query)
        query = "INSERT INTO users(id, name, email, email_verified_at, password, remember_token) VALUES "

In [ ]:
query = "INSERT INTO trips(start, end, amount, room_id, creator_id) VALUES "
for index in range(1, 500001):
    creator_id=random.randint(1, 300000)
    query += "('{}', '{}', {}, {}, {}),".format('2022-12-15 22:00:00', '2022-12-16 22:00:00', 1000, 20, creator_id)
    if index % 1000 == 0:
        query = query[:-1]
        await database.execute(query=query)
        query = "INSERT INTO trips(start, end, amount, room_id, creator_id) VALUES "

### Close the connection with the database

In [ ]:
await database.disconnect()